### Import 

In [1]:
from MFEA_lib.model import SMP_MFEA, MFEA_base, SA_MFEA, AbstractModel
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *
from MFEA_lib.operators.Search import *
from MFEA_lib.tasks.Benchmark.Competitions import CEC17_benchmark, GECCO20_benchmark_50tasks, WCCI22_benchmark

import os 
import sys 
import numpy as np 
import pandas as pd
import scipy 


In [2]:
primary_type= [int,np.int32,np.uint8, float, np.float32, np.float64, np.ndarray,tuple, bool, str, type(None), list] 

## Convert

### Convert .mso file 

In [8]:
# model = loadModel("C:/Users/Admin/Downloads/RESULT/Covert_ONLY_HISTORY/CEC17/COMPARE_KL_SBX_S-MFEA/SBX/S-MFEA_CEC17_PYTHON.mso", ls_tasks= CEC17_benchmark.get_10tasks_benchmark()[0]) 
model = loadModel("C:/Users/Admin/Downloads/RESULT/Covert_ONLY_HISTORY/GECCO/COMPARE_KL_GECCO_S-MFEA/S-MFEA_KL/1.mso", ls_tasks= WCCI22_benchmark.get_50tasks_benchmark(2)[0]) 



In [4]:
def process_list(ls): 
    global primary_type 
    new_ls = []
    for element in ls:
        if type(element) not in primary_type: 
            if type(element) is dict or hasattr(element, '__dict__'): 
               new_ls.append(convert_model(element, dict())) 

        else:
            if type(element) is list:
                new_ls.append(process_list(element))
            else: new_ls.append(element) 
    return new_ls

def process_dict(diction: dict, new_diction = dict()): 
    '''
    Return a dict 
    '''
    new_diction['name_type_object'] = 'dict' 
    # primary_type= [int, np.uint8, float, np.float32, np.float64, np.ndarray,tuple, bool, str, type(None), list] 
    global primary_type 

    for key, value in diction.items(): 
        if "method" in str(type(value)) or \
            "function" in str(type(value)) or \
            "attribute" in str(type(value)): 
            continue

        if hasattr(value, "__dict__"):
            new_diction[key] = convert_model(value, dict()) 
        else:
            if type(value) is list: 
                new_diction[key] = process_list(value)
            elif type(value) is dict: 
                new_diction[key] = process_dict(value, dict())
            elif type(value) in primary_type: 
                new_diction[key] = value 
            else: 
                print(f"{key} : {value}, {type(value)} cannot saved")
                # exit(0) 
                try:
                    new_diction[key] = str(value)
                except:
                    print(f"Cannot save {key} : {value}; type: {type(value)}")
    return new_diction 

def convert_model(model, new_model = dict()):
    '''
    Return a new model 
    '''
    new_model['name_type_object'] = type(model).__name__ if type(model).__name__ != "type" else model.__name__
    
    for key, value in model.__dict__.items(): 
        if hasattr(value, "__dict__"):
            new_model[key] = convert_model(value, dict())
        else: 
            if type(value) is list: 
                new_model[key] = process_list(value)
            elif type(value) is dict: 
                new_model[key] = process_dict(value, dict())
            elif type(value) in primary_type: 
                new_model[key] = value 
            else: 
                if "method" in str(type(value)) or \
                    "function" in str(type(value)) or \
                    "attribute" in str((value)) or \
                    "object" in str((value)): 
                    continue 
                print(f"=={key} : {value}, {type(value)}==cannot saved")
                try:
                    new_model[key] = str(value)
                except:
                    print(f"Cannot save {key} : {value}; type: {type(value)}")
    return new_model

In [9]:
type(model.ls_model[0].history_cost)

list

In [14]:
print(type(model.ls_model[0]))
print(type(model.ls_model[0]).__name__)

<class 'MFEA_lib.model.SMP_MFEA.model'>
model


In [5]:
new_model = convert_model(model, dict())
import pickle 
with open("new_model.mso", 'wb') as file: 
    pickle.dump(new_model, file)

** Problem: 
- \_\_name\_\_ modell in model.ls_model[0] cannot create because it is "model" :)

In [35]:
print(new_model.keys())

dict_keys(['name_type_object', 'model', 'name', 'list_attri_avg', 'ls_model', 'ls_seed', 'total_time', 'status', 'compile_kwargs', 'crossover', 'mutation', 'selection', 'search', 'attr_tasks', 'tasks', 'IndClass', 'args', 'kwargs', 'nb_generations', 'nb_inds_each_task', 'nb_inds_min', 'lr', 'p_const_intra', 'prob_search', 'lc_nums', 'nb_epochs_stop', 'swap_po', 'evaluate_initial_skillFactor', 'nb_run', 'history_cost', 'history_smp'])


In [36]:
convert_model(
    "C:/Users/Admin/Downloads/RESULT/Covert_ONLY_HISTORY/CEC17/COMPARE_KL_SBX_S-MFEA/SBX/S-MFEA_CEC17_PYTHON.mso",
    "", 
    ls_tasks= CEC17_benchmark.get_10tasks_benchmark()[0],
)

TypeError: convert_model() got an unexpected keyword argument 'ls_tasks'

In [ ]:
model.ls_model[0]

In [ ]:
convert_model()

In [ ]:
type((model.__dict__['history_cost']))

numpy.ndarray

In [ ]:
model.__dict__.keys()

dict_keys(['model', 'name', 'list_attri_avg', 'ls_model', 'ls_seed', 'total_time', 'status', 'compile_kwargs', 'crossover', 'mutation', 'selection', 'search', 'attr_tasks', 'tasks', 'IndClass', 'args', 'kwargs', 'nb_generations', 'nb_inds_each_task', 'nb_inds_min', 'lr', 'p_const_intra', 'prob_search', 'lc_nums', 'nb_epochs_stop', 'swap_po', 'evaluate_initial_skillFactor', 'nb_run', 'history_cost', 'history_smp'])

## Load


### Load new .mso file

In [5]:
import pickle 
import numpy as np 

with open("./new_model.mso", 'rb') as file: 
    model_dict = pickle.load(file)

# print(type(a))

In [9]:
def restore_list_attr(ls):
    global primary_type 
    # primary_type= [int, np.uint8, float, np.float32, np.float64, np.ndarray,tuple, bool, str, type(None), list]
    new_ls = [] 
    for element in ls:
        if type(element) in primary_type: 
            if type(element) is list:
                new_ls.append(restore_list_attr(element))
            else: new_ls.append(element) 
        else: 
            if type(element) is dict: 
                if element['name_type_object'] == 'dict': 
                    new_ls.append(restore_dict_attr(element))
                else:
                    new_ls.append(restore_from_dict(element))
    return new_ls

def restore_dict_attr(diction: dict): 
    new_dict = {}
    global primary_type
    # primary_type= [int, np.uint8, float, np.float32, np.float64, np.ndarray,tuple, bool, str, type(None), list]
    for key, value in diction.items():
        if type(value) is list:
            new_dict[key]= restore_list_attr(value) 
            pass 
        elif type(value) is dict: 
            if value['name_type_object'] == 'dict': 
                new_dict[key] = restore_dict_attr(value)
            else: 
                new_dict[key] = restore_from_dict(value) 
        elif type(value) in primary_type: 
            new_dict[key] = value 
        else: print(f"{key}: {value} cannot assigned")


def assign_attribute(model, attri_diction: dict):
    global primary_type
    for key, value in attri_diction.items():
        if type(value) is list: 
            # model.key = process_list_attr(value)
            setattr(model, key, restore_list_attr(value))
        elif type(value) is dict: 
            if value['name_type_object'] == 'dict': 
                setattr(model, key, restore_dict_attr(value))
            else:
                setattr(model, key, restore_from_dict(value))
            # temp_model = loadNewModel(value) 
            # model.key = temp_model
        elif type(value) in primary_type: 
            setattr(model, key, (value))
            # model.key = value 
        else: 
            print(f"{key}: {value} ||| {type(value)} cannot assigned")
        
    return model 


def restore_from_dict(diction):
    # Create model follow type "name_type_object"
    # primary_type= ["int", "np.uint8", "float", "np.float32", "np.float64", "np.float", "np.ndarray","tuple", "bool", "str", "NoneType"] 
    # primary_type = [str(type(x).__name__) for x in primary_type]
    print(diction['name_type_object'])
    class temp_class: 
        def __init__(self)->None:
            pass 

    model = temp_class

    new_model = assign_attribute(model(), diction)
    return new_model 


In [ ]:
new_model = restore_from_dict(model_dict)

In [ ]:
model_dict['ls_model']

In [15]:
new_

array([[1.10499036e+05, 1.11413430e+05, 1.09430201e+05, ...,
        1.13350167e+05, 1.11415546e+05, 1.09090756e+05],
       [1.09185515e+05, 1.09990245e+05, 1.08862274e+05, ...,
        1.12224713e+05, 1.10755671e+05, 1.08573066e+05],
       [1.07711844e+05, 1.09511944e+05, 1.08461749e+05, ...,
        1.11540641e+05, 1.08953806e+05, 1.07847828e+05],
       ...,
       [8.60070893e-01, 9.63130296e-01, 9.81602255e-01, ...,
        1.18659847e+00, 1.24658694e+00, 1.27807594e+00],
       [8.55662224e-01, 9.49325355e-01, 9.74076512e-01, ...,
        1.18058710e+00, 1.24373271e+00, 1.27271950e+00],
       [8.52230306e-01, 9.41964505e-01, 9.67978856e-01, ...,
        1.17683135e+00, 1.23542599e+00, 1.25938853e+00]])

In [11]:
crossover = getattr(new_model.ls_model[0], 'crossover')

In [14]:
print(crossover.name_type_object)

KL_SBXCrossover


In [ ]:
class Test: 
    def __init__(self) -> None:
        self.a = 1 
    def __init__(self, a)-> None:
        self.a = a  
    
a = Test
# a.b = 12
# print(a.b)
print(type(a))
print(a.__name__)

<class 'type'>
Test


In [ ]:
type(None)

NoneType